In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)

for i,layer in enumerate(network):
    print("U_",i+1)
    print(layer)
    

U_ 1
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
U_ 2
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


#### initialize_network(n_inputs, n_hidden, n_outputs):
initialize_network(2, 1, 2) -> 
3(2+bias 1)개의 input (layer층 = list), wieght 값은 random
2(1+bias 1)개의 hidden (layer층 = list), wieght값은 random
2개의 output layer (layer층 = list)

{input(x0,x1,x2)->hidden(z1) \
'weights': \
[U1_10 : 0.13436424411240122,\
U1_11 : 0.8474337369372327,\
U1_12 : 0.763774618976614]}\
{hidden(z0,z1) -> output(o1)\
'weights': \
[U2_10 : 0.2550690257394217, \
U2_11 : 0.49543508709194095]}\
{hidden(z0,z1) -> output(o2)\
'weights': \
[U2_20 : 0.4494910647887381, \
U2_21 : 0.651592972722763]}

In [2]:
network[1][-1]

{'weights': [0.4494910647887381, 0.651592972722763]}

In [3]:
def activate(weights, inputs):
    # print(weights, inputs)
    activation = weights[-1] #weight 글자 제외하고 값만 list로 반환
    #print(activation)
    for i in range(len(weights)-1):
        activation += inputs[i]*weights[i]  # 순전파 진행 
    return activation

def sigmoid(activation):
    activation = -1*activation
    return 1 / (1 + np.exp(activation)) # 시그모이드 구현

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs) 
            # print(sigmoid(activation))
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요? 아니용! 활성함수 거쳐야지이~
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠?
        inputs = new_inputs
    return inputs

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [4]:
network

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'weights': [0.2550690257394217, 0.49543508709194095]},
  {'weights': [0.4494910647887381, 0.651592972722763]}]]

forward propagation은 weight에 영향을 주지 않는다

In [5]:
row = [1, 0, -1]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [15]:
def sigmoid_derivative(output):
    return output*(1-output) # 시그모이드 미분

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:# 출력단이 아닌 경우
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        else:#출력단인 경우
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])
                # 역전파시 오차는 어떻게 설정했나요?
                #
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[i]*sigmoid_derivative(neuron['output']) # 시그모이드 함수를 사용한 역전파 

In [16]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [1.2761067582605141, -1.9164102920114108, -0.290011267847082], 'output': 0.9158042936008927, 'delta': 0.010943757640362116}, {'weights': [1.141085863492269, -1.8029866719781475, 0.160961391734561], 'output': 0.8904807883752178, 'delta': 0.013841676961410757}]
[{'weights': [2.212459179543436, 0.31699049750288, -1.1462829209469199], 'output': 0.7527649919102626, 'delta': 0.03705418908297403}, {'weights': [1.888282529986173, 1.0191338262091572, -1.20310315851855], 'output': 0.800901524996959, 'delta': 0.031747898832491565}]


delta 실제 값

In [46]:
from numpy import asarray
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate *neuron['delta']  # 퍼셉트론 학습 규칙
            
def mean_squared_error(y,t):
    sum = 0
    for i in range(len(y)):
        sum += (y[i]-t[i])**2
    return 0.5* (sum**0.5)         

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            expected = np.array(expected)
            outputs = np.array(outputs)
            sum_error += (1/2) * np.linalg.norm(expected - outputs, axis=0, ord=2)
            #print(mean_squared_error(expected,outputs))
            #sum_error += 1/2*sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            #sum_error += mean_squared_error(expected ,outputs) # 예측값의 오차 합
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [47]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [48]:
n_inputs = len(dataset[0])-1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
n_outputs = 2 # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? :class의 개수용!
network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.5 ,19 ,n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로,hint : for문))
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=3.824
>epoch=1, lrate=0.500, error=3.671
>epoch=2, lrate=0.500, error=3.616
>epoch=3, lrate=0.500, error=3.597
>epoch=4, lrate=0.500, error=3.590
>epoch=5, lrate=0.500, error=3.587
>epoch=6, lrate=0.500, error=3.585
>epoch=7, lrate=0.500, error=3.582
>epoch=8, lrate=0.500, error=3.577
>epoch=9, lrate=0.500, error=3.568
>epoch=10, lrate=0.500, error=3.551
>epoch=11, lrate=0.500, error=3.521
>epoch=12, lrate=0.500, error=3.471
>epoch=13, lrate=0.500, error=3.388
>epoch=14, lrate=0.500, error=3.300
>epoch=15, lrate=0.500, error=3.277
>epoch=16, lrate=0.500, error=3.281
>epoch=17, lrate=0.500, error=3.296
>epoch=18, lrate=0.500, error=3.324
[{'weights': [-0.9351774993622829, 0.8942275112432068, 0.7474733609442594], 'output': 0.0462341597674365, 'delta': -0.0073138782985926135}, {'weights': [-0.8232797416669756, 0.7287134132641059, 0.4898622990021188], 'output': 0.04743381649014043, 'delta': -0.0074942163254784365}]
[{'weights': [-0.4712757920492826, -0.13380707

In [56]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요?

In [57]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network,row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))

실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
